In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from annoy import AnnoyIndex

### Data Preparation

- Ran data_preparation.sql on spotify.sqlite and add 'tracks_info' table to the sql database

# Test run with subset of the dataset (10,000 tracks)

In [ ]:
data_path = 'data/spotify.sqlite'
conn = sqlite3.connect(data_path)
# conn.text_factory = lambda b: b.decode(errors = 'ignore')
cursor = conn.cursor()

In [3]:
query = '''
    SELECT *
    FROM tracks_info ti
'''
cursor.execute(query)
print("Query executed")

Query executed


In [4]:
index = AnnoyIndex(12, metric='angular')
index.set_seed(12)

In [ ]:
for i in range(0, 8740043):
    batch = cursor.fetchmany(1)[0][4:]
    index.add_item(i, batch)
    if i == 10000:
        break

In [6]:
index.build(10, n_jobs=-1)

In [6]:
names = []
values = []
i=1
for item in af:
    names.append(item[0])
    values.append(list(item[1:]))
    i+=1
    if i % 1000000 == 0:
        print(f"done with item {i}")


In [ ]:
data = {}
data['names'] = np.array(names)
data['vectors'] = np.array(values)

: 

: 

In [ ]:
data

{'names': ['2jKoVlU7VAmExKJ1Jh3w9P',
  '4JYUDRtPZuVNi7FAnbHyux',
  '6YjKAkDYmlasMqYw73iB0w',
  '2YlvHjDb4Tyxl4A1IcDhAe',
  '3UOuBNEin5peSRqdzvlnWM',
  '2g8HN35AnVGIk7B8yMucww',
  '7iL6o9tox1zgHpKUfh9vuC',
  '6KIKRz9eSTXdNsGUnomdtW',
  '33ZXjLCpiINn8eQIDYEPTD',
  '7N1Vjtzr1lmmCW9iasQ8YO',
  '1zzxoZVylsna2BQB65Ppcb',
  '4E5IFAXCob6QqZaJMTw5YN',
  '1Z4mVNtYGnpjHUoZVo3IMU',
  '0HNAPf0cLMkVQfwPl74kF3',
  '3gY6tiCNsuVi6s8kPV6aQg',
  '2wGSgTmgSF3xjRrHkTc25R',
  '5IweeQZdY5XEzStLzUJG5t',
  '67IvfvAMYQzJEeHopvwMMW',
  '1ktyqk3vVMB3zg17NjttnX',
  '1dizvxctg9dHEyaYTFufVi',
  '3jP6KQgQaL2JFEvBW6rELJ',
  '6MbxtmdQYE6eljl9sntLZt',
  '6Q3Fb6Co1NgIQ9njENgYSK',
  '0B2ZuTLZnWQ6gz8RlNwBu1',
  '0EZMXJMWf0tLKRWwCiA6Sx',
  '2p9sprAX27D6U5cuSE9NsM',
  '5iCm7z2e1IMPLA9gYiuv1P',
  '2I9foKseoFQh07p6sD2voE',
  '2ABUUwQ1E5gHkhk0uckzgo',
  '1ajsHJVZtBAnJcPxP2rhO8',
  '6Gpd4oUZL5jUpPR3mAirDc',
  '7Imv2O9gkwpZJcjKEcSqZ1',
  '4Bv9PjhIhp2sQE4S7dEAxR',
  '65zFbHuyUdai6WFwtPAkix',
  '2WwFItAZw5RcYv3rECNFIZ',
  '50QdRhmK

# Annoy

In [1]:
import annoy

ModuleNotFoundError: No module named 'annoy'

---

### (old)

In [10]:
af_df = pd.DataFrame(
    af, 
    columns=['id', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 
             'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']
    )

In [4]:
ids = tuple(af_features.id.values)

NameError: name 'af_features' is not defined

In [ ]:
query = f'SELECT * FROM tracks WHERE id IN {ids}'

tracks = pd.read_sql(query, conn).sort_values('id').loc[:, ['id', 'name']]

In [ ]:
query = f'SELECT * FROM r_track_artist WHERE track_id in {ids}'
track_artist = pd.read_sql(query, conn).rename(columns={'artist_id': 'id'})

In [ ]:
artist_ids = tuple(track_artist.artist_id.values)

In [ ]:
query = f'SELECT * FROM artists WHERE id in {artist_ids}'
artists = pd.read_sql(query, conn).loc[:, ['id', 'name']].set_index('id')

In [ ]:
# merge track_artist and artists
track_artist_name = track_artist.join(artists, on='id', how='left')

In [ ]:
track_artist_name = track_artist_name.groupby('track_id').agg({'name': '; '.join})

In [ ]:
df = af.join(tracks.set_index('id'), on='id', how='left').join(track_artist_name, on='id', how='left', lsuffix='_track', rsuffix='_artist')

In [ ]:
df.head(50)

In [176]:
# select a song to input

query = "SELECT * FROM tracks WHERE name = 'Girlfriend'"
input = pd.read_sql(query, conn)

In [177]:
input

,id,disc_number,duration,explicit,audio_feature_id,name,preview_url,track_number,popularity,is_playable
0,0VtrSyLoPpOIMlzfVPfbY7,1,217000,0,0VtrSyLoPpOIMlzfVPfbY7,Girlfriend,https://p.scdn.co/mp3-preview/1197cbe40fbfcd0d...,5,0,NaN
1,7wGOjwxFj8wbUwcu9rL47a,2,245253,0,7wGOjwxFj8wbUwcu9rL47a,Girlfriend,https://p.scdn.co/mp3-preview/b8a3d4b096b5ae09...,11,9,NaN
2,2uaTfYH3jfZ808xyeFif12,1,190240,0,2uaTfYH3jfZ808xyeFif12,Girlfriend,,9,17,NaN
3,6JqRzMVuX8sO5huQpG1vbi,1,171320,1,6JqRzMVuX8sO5huQpG1vbi,Girlfriend,https://p.scdn.co/mp3-preview/08a3942bf057ebbc...,5,45,NaN
4,63LozqLQXoviurS1xPV5FX,1,239746,0,63LozqLQXoviurS1xPV5FX,Girlfriend,https://p.scdn.co/mp3-preview/0558e52a24f1e962...,5,14,NaN
...,...,...,...,...,...,...,...,...,...,...
236,0hKR3GDXdzXQcXCkgFUNFa,1,72493,0,0hKR3GDXdzXQcXCkgFUNFa,Girlfriend,https://p.scdn.co/mp3-preview/cfbe28ffe31e8984...,10,3,NaN
237,0MsbKzHKuXEVUitS7e9nWa,1,187693,0,0MsbKzHKuXEVUitS7e9nWa,Girlfriend,,5,2,NaN
238,0mx99jEyxr83HLJ0y0JfKk,1,226000,0,0mx99jEyxr83HLJ0y0JfKk,Girlfriend,https://p.scdn.co/mp3-preview/594a75cdeb9882fa...,2,9,NaN
239,0nMEBbbohDmypdvpIKYqdw,1,165120,0,0nMEBbbohDmypdvpIKYqdw,Girlfriend,https://p.scdn.co/mp3-preview/9a6703d32b4985da...,3,4,NaN


In [92]:
# for index, row in af1.iterrows():
cos_sim = cosine_similarity(af1.iloc[:, 1:], af2.iloc[:, 1:])
af1['cos_sim'] = cos_sim
    # break

/tmp/ipykernel_22913/400334335.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  af1['cos_sim'] = cos_sim


In [93]:
af1.sort_values('cos_sim')

,id,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,cos_sim
51,2PxYJDohsBXaBLByHBITp5,0.955,0.366,0.0819,0.8880,5,0.0816,-18.674999,1,0.0437,49.112000,4,0.1210,0.967937
59,3jmUW1tGOpScXF5c0zXJOh,0.952,0.479,0.1790,0.9360,2,0.2080,-18.337000,1,0.0315,68.671997,3,0.2010,0.987620
47,5U6viBMb17ayuEad6Inpgg,0.980,0.462,0.1740,0.8720,3,0.1160,-18.377001,1,0.0365,69.302002,4,0.2120,0.988262
45,3WEkHcJisp1XyPPNMWKAi0,0.920,0.153,0.0186,0.3760,2,0.1070,-21.146000,1,0.0399,82.152000,4,0.1670,0.988622
94,6vZWpFl8wRQxQ69E9FoqZQ,0.996,0.462,0.2260,0.9480,3,0.1660,-16.773001,1,0.0642,66.658997,4,0.2200,0.989993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,5Ccyefda1pTeRCAg2kUIbX,0.983,0.429,0.2920,0.0700,6,0.3380,-12.436000,1,0.0315,95.543999,3,0.0734,0.999606
97,5bx9VS0wfBQfVYk0ujrBoB,0.996,0.616,0.3340,0.9460,8,0.2030,-17.108999,1,0.0473,130.572006,4,0.5610,0.999622
64,1DfwOz5apF0w2d8hccSkhH,0.996,0.575,0.2470,0.0684,9,0.3620,-13.284000,0,0.0400,130.804001,4,0.8410,0.999811
18,1ktyqk3vVMB3zg17NjttnX,0.847,0.639,0.6210,0.0208,8,0.3430,-9.981000,0,0.4030,92.582001,4,0.6000,0.999829


In [82]:
af1.iloc[:, 1:]

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.1800,0.893,0.514,0.000000,11,0.0596,-5.080000,1,0.2830,95.848000,4,0.787
1,0.2720,0.520,0.847,0.000000,9,0.3250,-5.300000,1,0.4270,177.371002,4,0.799
2,0.0783,0.918,0.586,0.000000,1,0.1450,-2.890000,1,0.1330,95.516998,4,0.779
3,0.5840,0.877,0.681,0.000000,1,0.1190,-6.277000,0,0.2590,94.834999,4,0.839
4,0.1700,0.814,0.781,0.000518,11,0.0520,-3.330000,1,0.2330,93.445000,4,0.536
...,...,...,...,...,...,...,...,...,...,...,...,...
94,0.9960,0.462,0.226,0.948000,3,0.1660,-16.773001,1,0.0642,66.658997,4,0.220
95,0.9950,0.472,0.223,0.980000,2,0.2990,-16.287001,1,0.0810,127.075996,4,0.372
96,0.9960,0.369,0.285,0.938000,7,0.1930,-16.523001,1,0.0528,129.557999,5,0.635
97,0.9960,0.616,0.334,0.946000,8,0.2030,-17.108999,1,0.0473,130.572006,4,0.561


In [90]:
af2.iloc[:, 1:]

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
99,0.996,0.477,0.319,0.972,11,0.148,-15.328,0,0.0542,134.102997,4,0.61
